In [1]:
import pyspark   
import pymongo
import pymongo_spark
from pymongo import MongoClient
from pyspark import SparkConf, SparkContext
pymongo_spark.activate()

#sc.stop()
conf = (SparkConf().set("spark.driver.maxResultSize", "2g"))
# 加大spark輸出結果用的記憶體大小(不是運算用的)

sc = pyspark.SparkContext(master="local[*]", conf=conf, appName="kmeans")

In [27]:
from pymongo import MongoClient
client = MongoClient('10.120.28.20',27017)
database = client['production']
collection =database['ebc'] #東森新聞407753筆

In [28]:
resp = collection.find({},{"tfidf":1,"_id":0})
print resp.count()

tfidf_list=[]
for post in resp:
    tfidf_list.append(" ".join(post.values()[0]))
    
#print tfidf_list[0:2]
print client.close()

407753
None


In [29]:
inputRDD = sc.parallelize(tfidf_list)


In [30]:
from pyspark.mllib.feature import HashingTF
hasingTF = HashingTF(2 ** 16)
trainTf = hasingTF.transform(inputRDD)
#trainTf.cache()

In [31]:
from pyspark.mllib.feature import IDF
idf = IDF().fit(trainTf)            # 訓練idf模型, 測試組不需要做訓練
trainTfidf = idf.transform(trainTf) # trainTfidf直接丟分群model作訓練

#trainTfidf.cache()
# help(idf) # 沒有看到idfmodel的保存方法...

In [32]:
trainTfidf.first() # SparseVector(文章, {關鍵詞1:向量, 關鍵詞2:向量, 關鍵詞3:向量,...})
                   # 都被hashing成數字代號了

SparseVector(65536, {2332: 5.2046, 2464: 4.9435, 2817: 4.9124, 3562: 5.1529, 3570: 3.3899, 4407: 4.1832, 6587: 0.7099, 7208: 5.7964, 7427: 1.8425, 7608: 3.6984, 8024: 2.996, 8468: 3.1793, 8813: 4.0813, 9568: 3.659, 10618: 7.1595, 10852: 2.1744, 11574: 5.188, 11893: 1.2611, 12321: 0.0, 12421: 1.2011, 12720: 2.3013, 12737: 5.527, 13847: 3.0551, 14308: 1.075, 14434: 6.4117, 14637: 2.3322, 16010: 2.6791, 17388: 4.4681, 17995: 3.338, 18481: 5.6935, 19290: 3.3089, 19969: 3.0692, 20134: 4.9336, 20468: 1.6358, 21720: 4.7875, 22770: 3.362, 24092: 4.0538, 25070: 12.8217, 25480: 1.9857, 26466: 4.8058, 27880: 2.1653, 31645: 5.155, 31660: 2.0422, 32266: 1.8338, 32758: 4.798, 33666: 1.2616, 34365: 6.5287, 34641: 5.6146, 39958: 5.6201, 40041: 1.633, 41344: 2.2827, 41407: 3.1055, 41809: 1.459, 42087: 2.8003, 42539: 2.5854, 42890: 2.5573, 43603: 13.7485, 43662: 1.451, 44006: 2.1388, 44210: 3.6316, 44653: 4.4371, 45074: 3.8441, 47168: 3.8244, 48457: 2.4429, 50021: 17.0355, 51079: 1.9736, 53654: 2.1397, 

In [16]:
# 給資料貼上標籤, 只有監督式學習才需要做此步驟, Kmeans(非監督式)跳過此步驟
from pyspark.mllib.regression import LabeledPoint

title_resp = collection.find({},{"title":1,"_id":0})
print client.close()

titleRDD = sc.parallelize(title_resp)
#print titleRDD.first()
trainLabels = titleRDD.map(lambda x:x.values()[0])
#print trainLabels.first()

#num = sc.parallelize(range(0,title_resp.count()))    # 產生流水號
#labelsDictRDD = trainLabels.zip(num)                 # 將RDD與流水號RDD組合變成(title,num)
                                                      # 不想用這方式作...

labelsDict = {}
title_resp = collection.find({},{"title":1,"_id":0})
print client.close()

i = 0
for title in title_resp:  # 製造(新聞標題:流水號)的字典給LabeledPoint方法使用
    labelsDict.__setitem__(title["title"],i)
    i += 1
    
train = trainLabels.zip(trainTfidf)\        # RDD.zip(RDD)將前後兩個RDD組合成一個RDD的方法
                  .map(lambda (title, vector): LabeledPoint(labelsDict[title], vector))
            #被LabeledPoint標記的項目可以用 lambda x: x.label 來查閱標記
print train.first()
#train.cache()

None
None
(0.0,(65536,[267,1643,2617,3314,5194,5440,5733,6091,6477,6587,7444,7446,9494,10143,10171,10670,12075,12321,12471,12781,15218,15410,16010,17436,17774,18156,18512,19310,19969,20489,20857,21062,21664,25740,25961,25967,27034,31819,32128,33039,34236,37756,38356,40650,41344,41455,42319,42600,42890,45536,45770,48113,48153,48304,49308,49740,49930,50086,50162,50525,50701,50767,51053,52723,53304,53809,54770,54776,58271,59108,59137,62233,64098],[1.17617052301,3.51154543883,2.75777363645,1.77694438344,3.51154543883,2.11488829068,2.08442908319,4.6101577275,15.8792880081,0.765343471764,2.48490664979,1.74795684657,3.5102506721,11.5317471176,1.87741491381,6.44772673276,4.74368912012,0.0,4.05054193956,4.29170399638,3.07868135653,2.86095787269,1.31432086149,3.05201310945,4.07127783804,7.19711363164,1.50221346334,1.87741491381,0.720891709194,4.59030022901,4.94018312801,4.49237469184,3.02603762305,3.05201310945,5.36453216789,1.64617415226,2.32015141671,2.33289044249,3.51154543883,3.74516029001,2

In [19]:
lab = train.map(lambda x:x.label) # 查閱label
lab.count()
lab.take(5)

[0.0, 1.0, 2.0, 3.0, 4.0]

In [34]:
from pyspark.mllib.clustering import KMeans
from pyspark.mllib.clustering import KMeansModel
from numpy import array

c = KMeans.train(trainTfidf, 10, maxIterations=10,  
        runs=1, initializationMode="random")
#訓練組RDD, K=10群, 疊代=10次, 跑1輪, 起始點模式="隨機"

c.save(sc, "file:/home/cloudera/Desktop/text_mining/KMeansModel") #保存訓練好的模型

# 以下是回測訓練組才做的
#result = c.predict(trainTfidf) 
#print result.count()
#print result.first()
#k_data = array(result.collect())
#print array(k_data)
# c = KMeansModel.load(sc, "file:/home/cloudera/Desktop/text_mining/KMeansModel")

In [37]:
import time
start = time.time()
collection2 =database['20130707']     # 2013年7月7號當天的所有新聞803筆當作測試組
resp = collection2.find({},{"tfidf":1,"_id":0})
print resp.count()

tfidf_list=[]
for post in resp:
    tfidf_list.append(" ".join(post.values()[0]))

client.close()

inputRDD = sc.parallelize(tfidf_list)
testTf = hasingTF.transform(inputRDD)
testTfidf = idf.transform(testTf)

# 讀取之前用40萬筆資料訓練的kmeans模型
km = KMeansModel.load(sc, "file:/home/cloudera/Desktop/text_mining/KMeansModel") 
result = km.predict(testTfidf)
k_data = array(result.collect())
print k_data # 模型分類10組, 輸出的數字是每一篇新聞被分到第幾組
print "資料分組耗時", time.time()-start, "秒"

803
[7 7 7 7 7 6 3 7 7 7 1 7 1 7 7 7 7 7 0 2 7 7 7 5 6 7 6 7 8 5 2 7 7 7 7 7 7
 6 8 0 7 7 2 6 8 0 7 5 5 5 7 3 6 6 5 5 5 3 5 5 5 7 5 5 5 7 7 7 7 0 6 7 0 7
 6 7 7 7 0 5 7 7 7 6 7 7 6 7 2 7 7 7 7 6 7 6 9 7 6 6 6 7 6 6 7 7 7 7 6 7 6
 5 7 7 5 7 7 8 0 6 7 6 7 7 5 7 7 8 5 6 3 6 3 8 3 6 3 3 4 3 3 6 8 3 6 3 7 3
 7 4 7 7 7 7 3 7 3 0 6 5 6 5 5 8 6 6 8 7 0 6 6 7 7 5 0 6 6 0 0 0 0 6 3 0 7
 7 0 7 6 7 7 8 7 0 7 8 0 0 7 7 0 0 7 7 0 7 7 7 1 7 7 7 7 7 7 7 7 7 5 5 5 5
 5 7 7 5 7 5 5 5 7 5 5 7 7 7 5 5 5 4 5 5 5 5 5 7 7 7 7 7 7 7 7 7 7 6 7 7 7
 7 7 7 7 7 7 7 5 7 7 2 5 2 5 5 5 7 7 7 7 7 7 7 7 0 7 5 0 6 7 5 5 7 5 7 5 5
 5 5 7 7 5 5 5 3 5 7 7 6 7 7 7 9 1 7 5 7 7 7 7 7 7 5 5 5 6 7 5 5 5 7 7 3 7
 5 7 7 7 5 7 5 5 5 5 7 5 7 7 2 7 7 5 7 5 5 5 3 5 5 7 6 5 5 5 7 5 5 5 3 3 3
 3 4 3 7 3 4 0 4 6 7 7 7 7 6 6 2 7 7 7 8 5 4 7 7 5 7 7 7 6 7 7 9 3 7 3 3 7
 7 6 5 3 5 3 4 7 3 7 7 7 4 7 3 3 4 3 3 3 3 5 3 6 6 7 3 4 6 7 3 7 7 7 4 6 7
 7 5 7 7 3 7 7 5 7 3 7 5 6 7 3 2 7 7 7 3 7 3 4 3 7 6 7 7 0 7 5 7 5 0 2 7 7
 5 6 6 7 3 7 3 7 0 3 

In [ ]:
# 官方文件範例
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

# Load and parse the data
data = sc.textFile("data/mllib/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10,
        runs=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
clusters.save(sc, "myModelPath")
sameModel = KMeansModel.load(sc, "myModelPath")